In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Adding an Objective

To add an objective to an optimization, use the `add_objective` method on System.

```{eval-rst}
    .. automethod:: openmdao.core.system.System.add_objective
        :noindex:
```

## Specifying units

You can specify units when adding an objective. When this is done, the quanitity is converted from the target output’s units to the desired unit before giving it to the optimizer. If you also specify scaling, that scaling is applied after the unit conversion. Moreover, the upper and lower bound should be specified using these units.

In [ ]:
import openmdao.api as om

prob = om.Problem()
model = prob.model

model.add_subsystem('comp1', om.ExecComp('y1 = 2.0*x',
                                         x={'val': 2.0, 'units': 'degF'},
                                         y1={'val': 2.0, 'units': 'degF'}),
                    promotes=['x', 'y1'])

model.add_subsystem('comp2', om.ExecComp('y2 = 3.0*x',
                                         x={'val': 2.0, 'units': 'degF'},
                                         y2={'val': 2.0, 'units': 'degF'}),
                    promotes=['x', 'y2'])

model.set_input_defaults('x', 35.0, units='degF')

model.add_design_var('x', units='degC', lower=0.0, upper=100.0)
model.add_constraint('y1', units='degC', lower=0.0, upper=100.0)
model.add_objective('y2', units='degC')

prob.setup()
prob.run_driver()

In [ ]:
print('Model variables')
print(prob.get_val('x', indices=[0]))

In [ ]:
print(prob.get_val('comp2.y2', indices=[0]))

In [ ]:
print(prob.get_val('comp1.y1', indices=[0]))

In [ ]:
print('Driver variables')
dv = prob.driver.get_design_var_values()
print(dv['x'][0])

In [ ]:
obj = prob.driver.get_objective_values(driver_scaling=True)
print(obj['y2'][0])

In [ ]:
con = prob.driver.get_constraint_values(driver_scaling=True)
print(con['y1'][0])

In [ ]:
import numpy as np
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('x', indices=[0]), 35.)
assert_near_equal(prob.get_val('comp2.y2', indices=[0]), 105.)
assert_near_equal(prob.get_val('comp1.y1', indices=[0]), 70.)
assert_near_equal(dv['x'][0], 1.6666666666666983)
assert_near_equal(obj['y2'][0], 40.555555555555586)
assert_near_equal(con['y1'][0], 21.111111111111143)